In [7]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import random

## Creating the function

In [8]:
# Auxillary Function provided to get ticker price
def getTickerPrice(ticker: str, date: np.datetime64) -> float:
    return random.uniform(1, 100)

In [9]:
# main function to calculate or backtest trade performance
def calculateTradePerformance(df):
    metrics = {} #Declaring a dictionary
    
        # keeping the columns we want to use and drop any column that we don't require
    df = df[['Date', 'Symbol', 'Side', 'Size', 'Price']].copy()
   
        # drop rows where price is not provided
    df.dropna(subset = ['Price', 'Size'], inplace = True)
    
        # if dataframe is empty return everything as zero in dictionary
    if df.empty:
        return 'DataFrame empty!'
    
        # calculating current price using the aux function
    df['Current Price'] = df.apply(lambda row: getTickerPrice(row['Symbol'], row['Date']), axis=1)
    
        # creating a column called Profits using the np.where()
    df['Profits'] = np.where(df['Side'] == 'buy', #this is boolean condition
                            df['Size'] * (df['Current Price'] - df['Price']), #it will execute this if above condition is True
                            df['Size'] * (df['Price'] - df['Current Price'])) #else this will be executed
    
        # duplicating the df for winning_trades and losing_tradesz
    winning_trades = df[df['Profits']>0]
    losing_trades = df[df['Profits']<0]
    
        # Calculating the mtrics
    # 1. Average Loser
    metrics['Avg Loser'] = losing_trades['Profits'].mean() if not losing_trades.empty else 0
    
    # 2. Gross return
    df['Amount Invested'] = df['Size'] * df['Price']
    df['Revenue'] = df['Amount Invested'] + df['Profits']
        # now calculating gross return
    total_amount_invested = df['Amount Invested'].sum()
    total_revenue = df['Revenue'].sum()
    metrics['gross_return'] = ((total_revenue - total_amount_invested)/total_amount_invested)*100
    
    # 3. Average Profits Per Trade
    metrics['Average Profits/Trade'] = df['Profits'].mean()
    
    # 4. Reward to Risk Ratio
    metrics['Reward/Risk'] = total_revenue / total_amount_invested
    
    # 5. Win Rate and Lose Rate
    metrics['Win Rate'] = len(winning_trades) / len(df)
    metrics['Lose Rate'] = 1 - metrics['Win Rate']
    
    # 6. Sharpe Ratio
    average_return = df['Profits'].mean()
    risk_free_rate = 0.01  # Assuming risk-free rate is 0.01
    standard_deviation_return = df['Profits'].std()
    metrics['Sharpe Ratio'] = (average_return - risk_free_rate) / standard_deviation_return if standard_deviation_return != 0 else 0
    
    # 7. Total Winning Trades and Total Losing Trades
    metrics['Total Winning Trades'] = len(winning_trades)
    metrics['Total Losing Trades'] = len(losing_trades)
    
    # 8. Largest Winning Trade
    metrics['Largest Winning Trade'] = winning_trades['Profits'].max() if not winning_trades.empty else 0
    
    # 9. Largest Losing Trade
    metrics['Largest Losing Trade'] = losing_trades['Profits'].min() if not losing_trades.empty else 0
    
    # 10. Annual Rate of Returns
        # This takes the max (recent date at which a trade was taken) and subtracts with min (1st trade taken in the dataset) to calculate total days.
    num_years = (df['Date'].max() - df['Date'].min()).days / 365 
        # ending_value and starting_value was calculated in gross return
    metrics['Annual Rate of Returns'] = ((total_revenue / total_amount_invested) ** (1 / num_years) - 1) if num_years > 0 else 0
    
    return metrics

# Trial on a dataset

In [10]:
# Reading or loading the given Dataset
df = pd.read_csv("testData.csv")
df

,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol
0,2023,6/15/2023,5/20/2023,Spouse,AAPL,Apple Inc. Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,CA-12,Yes,NaN
1,2023,5/12/2023,4/10/2023,Self,GOOGL,Alphabet Inc. Stock,Sale (Full),"$50,001 - $100,000",Nancy Pelosi,CA-12,No,NaN
2,2023,12/29/2023,12/6/2022,Dependent,AMZN,AMZN Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,NY-14,Yes,NaN
3,2023,2/13/2022,4/14/2022,Self,AMZN,AMZN Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,FL-9,No,NaN
4,2023,6/23/2023,3/11/2022,Joint,MSFT,MSFT Stock,Purchase,"$1,001 - $15,000",Nancy Pelosi,CA-12,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
97,2023,3/15/2023,7/26/2023,Dependent,AAPL,AAPL Stock,Sale (Full),"$1,001 - $15,000",Nancy Pelosi,FL-9,Yes,NaN
98,2022,6/8/2022,5/3/2023,Joint,TSLA,TSLA Stock,Sale (Full),"$50,001 - $100,000",Nancy Pelosi,CA-12,No,NaN
99,2023,7/24/2022,9/16/2023,Self,GOOGL,GOOGL Stock,Sale (Partial),"$1,001 - $15,000",Nancy Pelosi,TX-7,No,NaN
100,2023,12/25/2022,6/25/2022,Spouse,AAPL,AAPL Stock,Sale (Full),"$100,001 - $250,000",Nancy Pelosi,NY-14,Yes,NaN


## What I observe in the dataset is that there is no price, no size and no clear side columns.
## What I will do in this case is to assume the amount column depicts the amount of money invested. So I will take the average of the range of amount provided in each row.
## And using the aux function (getTickerPrice()) make a price column. Using the price column and the avg amount of money I will calculate the Size for each trade.
## Also I will consider Purchase as Buy and Sale as Sell.

In [11]:
# Pre-Processing the dataset
    # Renaming the columns to suit for the function
df = df.rename(columns = {'transactionDate' : 'Date', 'ticker' : 'Symbol', 'type' : 'Side'})

    # Converting the Date column to datetime format
df['Date']= pd.to_datetime(df['Date'])

    # The dataset was mixed so sorted it in ascending order w.r.t Date column
df = df.sort_values(by = 'Date')

    # Mapping the 'Side' values, Purchase as buy and Sale as sell
df['Side'] = df['Side'].map({'Purchase' : 'buy', 'Sale (Full)' : 'sell', 'Sale (Partial)' : 'sell'})

    # Making a price column using the aux function
df['Price'] = df.apply(lambda row: getTickerPrice(row['Symbol'], row['Date']), axis=1)

    # The amount column was a single string so split it into 2 diff strings, these 2 diff strings can later be used to convert into int or float
df['amount'] = df['amount'].str.replace('$', '').str.replace(',', '').str.split(' - ')

    # Creating amoun invested column by calculating the average of each row of amount column
df['Amount Invested'] = df['amount'].apply(lambda x: (float(x[0]) + float(x[1])) / 2)

    # Making Size column using the amount invested and price column. Rounding it off to 2 decimals
df['Size'] = round(df['Amount Invested'] / df['Price'], 2)

df

,disclosureYear,disclosureDate,Date,owner,Symbol,assetDescription,Side,amount,representative,district,capitalGainsOver200USD,option_symbol,Price,Amount Invested,Size
21,2022,4/4/2023,2022-01-08,Self,MSFT,MSFT Stock,sell,"[15001, 50000]",Nancy Pelosi,CA-12,No,NaN,84.645386,32500.5,383.96
31,2022,12/31/2022,2022-02-09,Joint,AAPL,AAPL Stock,sell,"[1001, 15000]",Nancy Pelosi,CA-12,No,NaN,60.900786,8000.5,131.37
48,2022,4/16/2023,2022-02-27,Spouse,AAPL,AAPL Stock,sell,"[250001, 500000]",Nancy Pelosi,NY-14,No,NaN,40.436611,375000.5,9273.79
87,2022,7/3/2022,2022-03-07,Joint,AAPL,AAPL Stock,sell,"[250001, 500000]",Nancy Pelosi,FL-9,Yes,NaN,61.215687,375000.5,6125.89
8,2023,4/17/2023,2022-03-10,Spouse,AMZN,AMZN Stock,buy,"[1001, 15000]",Nancy Pelosi,NY-14,Yes,NaN,83.373412,8000.5,95.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2023,5/11/2023,2023-12-11,Dependent,MSFT,MSFT Stock,sell,"[50001, 100000]",Nancy Pelosi,TX-7,Yes,NaN,83.622815,75000.5,896.89
55,2022,4/13/2022,2023-12-11,Self,GOOGL,GOOGL Stock,sell,"[1001, 15000]",Nancy Pelosi,TX-7,No,NaN,61.779880,8000.5,129.50
44,2022,2/11/2022,2023-12-12,Dependent,AAPL,AAPL Stock,buy,"[250001, 500000]",Nancy Pelosi,TX-7,Yes,NaN,23.093590,375000.5,16238.29
68,2023,10/7/2022,2023-12-18,Joint,GOOGL,GOOGL Stock,buy,"[250001, 500000]",Nancy Pelosi,NY-14,No,NaN,63.394977,375000.5,5915.30


In [12]:
metrics = calculateTradePerformance(df)
metrics

{'Avg Loser': -219923.65205074978,
 'gross_return': 24.128060683963596,
 'Average Profits/Trade': 30931.345370529812,
 'Reward/Risk': 1.241280606839636,
 'Win Rate': 0.5098039215686274,
 'Lose Rate': 0.4901960784313726,
 'Sharpe Ratio': 0.04123824448169231,
 'Total Winning Trades': 52,
 'Total Losing Trades': 50,
 'Largest Winning Trade': 4373172.320451675,
 'Largest Losing Trade': -4812847.057520552,
 'Annual Rate of Returns': 0.11580129190530286}

## As we have taken price and current_price using random function. The returns might look wrong. But if we provide correct details like size, price, current_price we can expect a correct output.